In [1]:
from gutenberg_catalog.sources import BookXML

In [2]:
b = BookXML.from_file('cache/epub/2600/pg2600.rdf')

In [4]:
dict(b)

{'id': 2600,
 'title': 'War and Peace',
 'creators': [[('deathdate', '1910'),
   ('name', 'Tolstoy, Leo, graf'),
   ('alias', 'Tolstoï, Léon'),
   ('birthdate', '1828'),
   ('alias', 'Толстой, Лев Николаевич'),
   ('alias', 'Tolstoi, Leo'),
   ('alias', 'Tolstoy, Graf Leo'),
   ('alias', 'Tolstoy, Lev N.'),
   ('webpage', 'http://en.wikipedia.org/wiki/Leo_Tolstoy')]],
 'author': 'Tolstoy, Leo, graf',
 'surname': 'Tolstoy',
 'subjects': ['Aristocracy (Social class) -- Russia -- Fiction',
  'PG',
  'Napoleonic Wars, 1800-1815 -- Campaigns -- Russia -- Fiction',
  'War stories',
  'Historical fiction',
  'Russia -- History -- Alexander I, 1801-1825 -- Fiction'],
 'formats': [{'url': 'http://www.gutenberg.org/ebooks/2600.epub.noimages',
   'formats': ['application/epub+zip'],
   'extent': 1342035},
  {'url': 'http://www.gutenberg.org/ebooks/2600.kindle.images',
   'formats': ['application/x-mobipocket-ebook'],
   'extent': 5470783},
  {'url': 'http://www.gutenberg.org/ebooks/2600.kindle.no

In [5]:
b.to_json()

'{"id":2600,"title":"War and Peace","creators":[[["deathdate","1910"],["name","Tolstoy, Leo, graf"],["alias","Tolsto\\u00ef, L\\u00e9on"],["birthdate","1828"],["alias","\\u0422\\u043e\\u043b\\u0441\\u0442\\u043e\\u0439, \\u041b\\u0435\\u0432 \\u041d\\u0438\\u043a\\u043e\\u043b\\u0430\\u0435\\u0432\\u0438\\u0447"],["alias","Tolstoi, Leo"],["alias","Tolstoy, Graf Leo"],["alias","Tolstoy, Lev N."],["webpage","http:\\/\\/en.wikipedia.org\\/wiki\\/Leo_Tolstoy"]]],"author":"Tolstoy, Leo, graf","surname":"Tolstoy","subjects":["Aristocracy (Social class) -- Russia -- Fiction","PG","Napoleonic Wars, 1800-1815 -- Campaigns -- Russia -- Fiction","War stories","Historical fiction","Russia -- History -- Alexander I, 1801-1825 -- Fiction"],"formats":[{"url":"http:\\/\\/www.gutenberg.org\\/ebooks\\/2600.epub.noimages","formats":["application\\/epub+zip"],"extent":1342035},{"url":"http:\\/\\/www.gutenberg.org\\/ebooks\\/2600.kindle.images","formats":["application\\/x-mobipocket-ebook"],"extent":547078